<a href="https://colab.research.google.com/github/sahandfrd94/NLP_task_2_sentimental-analysis/blob/main/sentimental_analysis_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#import dataset
##Tweitter dataset

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list
!kaggle datasets download -d kazanova/sentiment140


In [2]:
! unzip -qx /content/sentiment140.zip

#preprocessing data


In [4]:
!head -5 /content/training.1600000.processed.noemoticon.csv

"0","1467810369","Mon Apr 06 22:19:45 PDT 2009","NO_QUERY","_TheSpecialOne_","@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
"0","1467810672","Mon Apr 06 22:19:49 PDT 2009","NO_QUERY","scotthamilton","is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"
"0","1467810917","Mon Apr 06 22:19:53 PDT 2009","NO_QUERY","mattycus","@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds"
"0","1467811184","Mon Apr 06 22:19:57 PDT 2009","NO_QUERY","ElleCTF","my whole body feels itchy and like its on fire "
"0","1467811193","Mon Apr 06 22:19:57 PDT 2009","NO_QUERY","Karoli","@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. "


In [5]:
clm = ['target' , 'id number' , 'date and time' , 'query' ,'user', 'content']

In [7]:
import pandas as pd
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names = clm , encoding='latin-1')
df.head()

,target,id number,date and time,query,user,content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   target         1600000 non-null  int64 
 1   id number      1600000 non-null  int64 
 2   date and time  1600000 non-null  object
 3   query          1600000 non-null  object
 4   user           1600000 non-null  object
 5   content        1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [9]:
df['target'].unique()

array([0, 4])

In [10]:
sentence = list(df['content'])
label = df['target']
label[label == 4] = 1

<ipython-input-10-f855d4fad74e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label[label == 4] = 1


#create word embedding

In [12]:
vocab_size = 100000
max_len = 32
oov = '<oov>'
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = vocab_size , oov_token = oov)
tokenizer.fit_on_texts(sentence)

seq = tokenizer.texts_to_sequences(sentence)

from tensorflow.keras.preprocessing.sequence import pad_sequences
pad = pad_sequences(seq , maxlen = max_len ,
                    padding='post',
                    truncating = 'post'
)


In [13]:
#split train and test data
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test=train_test_split(pad, label ,
                                                test_size = 0.2,random_state = 101)

LSTM

In [15]:

from keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Dropout
lstm = Sequential()
lstm.add(Embedding(input_dim = vocab_size,output_dim=100,input_length = max_len))
lstm.add(LSTM(128 ,return_sequences=True))
lstm.add(LSTM(64))
lstm.add(Dense(64,activation = 'relu'))
lstm.add(Dropout(0.3))
lstm.add(Dense(1,activation = 'sigmoid'))

In [16]:
lstm.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [ ]:
lstm.summary()

In [18]:
lstm.fit(
    x_train,y_train,
    epochs = 25,validation_data = [x_test,y_test],
    verbose = 1
)

Epoch 1/25
40000/40000 [==============================] - 454s 11ms/step - loss: 0.4088 - accuracy: 0.8145 - val_loss: 0.3775 - val_accuracy: 0.8295
Epoch 2/25
40000/40000 [==============================] - 423s 11ms/step - loss: 0.3471 - accuracy: 0.8482 - val_loss: 0.3717 - val_accuracy: 0.8336
Epoch 3/25
40000/40000 [==============================] - 422s 11ms/step - loss: 0.3113 - accuracy: 0.8674 - val_loss: 0.3863 - val_accuracy: 0.8286
Epoch 4/25
40000/40000 [==============================] - 428s 11ms/step - loss: 0.2773 - accuracy: 0.8843 - val_loss: 0.4023 - val_accuracy: 0.8241
Epoch 5/25
40000/40000 [==============================] - 423s 11ms/step - loss: 0.2463 - accuracy: 0.8988 - val_loss: 0.4203 - val_accuracy: 0.8198
Epoch 6/25
40000/40000 [==============================] - 418s 10ms/step - loss: 0.2197 - accuracy: 0.9111 - val_loss: 0.4562 - val_accuracy: 0.8146
Epoch 7/25
40000/40000 [==============================] - 419s 10ms/step - loss: 0.1981 - accuracy: 0.9205